# Get own implied volatilities and compare them to Deribits

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns
import plotly.graph_objects as go
import datetime
from scipy import stats as sps

from utils import get_human_timestamp, get_difference_between_now_and_expirety_date
import plotly
import plotly.graph_objs as go
from typing import Union
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv(f"./datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
data = df.copy()

In [27]:
# Ньютон-Рафсон
def get_implied_volatility_call(
    option_type: Union["call", "put"],
    C: float,
    K: float,
    T: float,
    F: float,
    r: float = 0.0,
    error: float = 0.001,
) -> float:
    """
    Function to count implied volatility via given params of option, using Newton-Raphson method :

    Args:
        C (float): Option market price(USD).
        K (float): Strike(USD).
        T (float): Time to expiration in years.
        F (float): Underlying price.
        r (float): Risk-free rate.
        error (float): Given threshhold of error.

    Returns:
        float: Implied volatility in percent.
    """
    vol = 1.0
    dv = error + 1
    while abs(dv) > error:
        d1 = (np.log(F / K) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        D = np.exp(-r * T)
        if option_type.lower() == "call":
            price = F * sps.norm.cdf(d1) - K * sps.norm.cdf(d2) * D
        elif option_type.lower() == "put":
            price = -F * sps.norm.cdf(-d1) + K * sps.norm.cdf(-d2) * D
        else:
            raise ValueError("Wrong option type, must be 'call' or 'put' ")
        Vega = F * np.sqrt(T / np.pi / 2) * np.exp(-0.5 * d1**2)
        PriceError = price - C
        dv = PriceError / Vega
        vol = vol - dv
    # in percents, as deribit shows
    return vol * 100

In [50]:
test = data.sample(10)
test[["symbol", "type", "mark_price", "underlying_price", "strike_price", "expiration", "delta"]]

,symbol,type,mark_price,underlying_price,strike_price,expiration,delta
10610035,BTC-31MAR23-100000-P,put,4.8531,17086.6200,100000,1680249600000000,-0.99552
2624467,ETH-1DEC22-1250-C,call,0.0286,1286.0339,1250,1669881600000000,0.92876
2894662,ETH-29SEP23-2000-P,put,0.6851,1285.4500,2000,1695974400000000,-0.61943
12055689,BTC-23DEC22-16000-C,call,0.0971,17198.3000,16000,1671782400000000,0.71666
4836775,BTC-29SEP23-21000-C,call,0.1612,17139.7300,21000,1695974400000000,0.48145
10203237,BTC-29SEP23-23000-P,put,0.4757,17096.8200,23000,1695974400000000,-0.58419
19373647,ETH-30JUN23-3000-P,put,1.3879,1273.3800,3000,1688112000000000,-0.86680
20771439,BTC-29SEP23-17000-P,put,0.2402,16928.5300,17000,1695974400000000,-0.38388
14130081,SOL-30DEC22-36-P,put,1.7426,13.1441,36,1672387200000000,-0.96968
2141430,ETH-27JAN23-1300-P,put,0.1248,1286.9100,1300,1674806400000000,-0.45393


# In USD

In [51]:
for index, row in test.iterrows():
    iv = get_implied_volatility_call(
        option_type = row["type"], 
        C = row["mark_price"]*row["underlying_price"],
        K = row["strike_price"],
        T = get_difference_between_now_and_expirety_date(row["expiration"], row["timestamp"]),
        F = row["underlying_price"],
        r = 0.0,
        error = 0.001
    )
    deribit_value = row["mark_iv"]
    print(f"""Deribit IV: {deribit_value}, counted IV: {iv}, error: {2*(deribit_value - iv)/(iv + deribit_value) * 100}% """)

Deribit IV: 105.73, counted IV: 103.49751519757629, error: 2.134026014996687% 
Deribit IV: 80.85, counted IV: 79.53825673653012, error: 1.6357098582655962% 
Deribit IV: 75.19, counted IV: 75.18734067702545, error: 0.0035368666084647743% 
Deribit IV: 59.05, counted IV: 59.02858912700836, error: 0.036265462095939324% 
Deribit IV: 65.13, counted IV: 65.1354161934405, error: -0.008315627583713004% 
Deribit IV: 64.48, counted IV: 64.48164341490912, error: -0.0025486879130777822% 
Deribit IV: 79.79, counted IV: 79.77831251050685, error: 0.01464888524453876% 
Deribit IV: 66.52, counted IV: 66.53352332362292, error: -0.020327644522463773% 
Deribit IV: 169.92, counted IV: nan, error: nan% 
Deribit IV: 75.54, counted IV: 75.63408206088228, error: -0.12446850624088096% 


# In crypto

In [52]:
for index, row in test.iterrows():
    iv = get_implied_volatility_call(
        option_type = row["type"], 
        C = row["mark_price"],
        K = row["strike_price"]/row["underlying_price"],
        T = get_difference_between_now_and_expirety_date(row["expiration"], row["timestamp"]),
        F = row["underlying_price"]/row["underlying_price"], # always 1.0
        r = 0.0,
        error = 0.001
    )
    deribit_value = row["mark_iv"]
    print(f"""Deribit IV: {deribit_value}, counted IV: {iv}, error: {2*(deribit_value - iv)/(iv + deribit_value) * 100}% """)

Deribit IV: 105.73, counted IV: 103.49751519757986, error: 2.1340260149932413% 
Deribit IV: 80.85, counted IV: 79.5382567365327, error: 1.6357098582623446% 
Deribit IV: 75.19, counted IV: 75.18734067702549, error: 0.0035368666084080727% 
Deribit IV: 59.05, counted IV: 59.02858912700834, error: 0.036265462095975434% 
Deribit IV: 65.13, counted IV: 65.13541619344052, error: -0.008315627583734822% 
Deribit IV: 64.48, counted IV: 64.48164341490906, error: -0.002548687912989628% 
Deribit IV: 79.79, counted IV: 79.77831251050691, error: 0.014648885244467506% 
Deribit IV: 66.52, counted IV: 66.53352332362292, error: -0.020327644522463773% 
Deribit IV: 169.92, counted IV: nan, error: nan% 
Deribit IV: 75.54, counted IV: 75.63408206088224, error: -0.12446850624082459% 
